### The following is a WIP and incomplete

In [1]:
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import json
from bs4 import BeautifulSoup
import re
import string
from nltk import corpus, WordNetLemmatizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [2]:
# Read in the JSON structure using a for loop so as to read in each line one at a time
with open('C:/Users/Vikram Pande/Desktop/ads-50k.json', encoding='utf-8') as f:
    d = [json.loads(line) for line in f]

In [31]:
# Convert to a flat Pandas Data Frame, separating the nested JS Objects (especially in the 'metadata' object) into 
# separate columns
df = json_normalize(d)
df

,id,title,abstract,content,metadata.standout.bullet1,metadata.standout.bullet2,metadata.standout.bullet3,metadata.additionalSalaryText,metadata.classification.name,metadata.subClassification.name,metadata.location.name,metadata.workType.name,metadata.area.name,metadata.suburb.name
0,38915469,Recruitment Consultant,We are looking for someone to focus purely on ...,<HTML><p>Are you looking to join a thriving bu...,Join a Sector that is considered Recession Pro...,Excellent opportunity for Career Progression ...,Make a Diference whilst earning Money and havi...,commission,Education & Training,Other,Sydney,Full Time,NaN,NaN
1,38934839,Computers Salesperson - Coburg,Passionate about exceptional customer service?...,<HTML><p>&middot;&nbsp;&nbsp;Casual hours as r...,NaN,NaN,NaN,Attractive Commission - Uncapped Earning Poten...,Retail & Consumer Products,Retail Assistants,Melbourne,Casual/Vacation,Northern Suburbs,Coburg
2,38946054,Senior Developer | SA,Readifarians are known for discovering the lat...,<HTML><p>Readify helps organizations innovate ...,"Design, develop, test and deliver custom softw...",Keep your skills current with 20 x paid profes...,Flexible & inclusive work environment,NaN,Information & Communication Technology,Consultants,Adelaide,Full Time,NaN,NaN
3,38833950,Senior Commercial Property Manager | Leading T...,~ Rare opportunity for a Senior PM to step int...,<HTML><p><strong>WayPoint Recruitment&nbsp;</s...,NaN,NaN,NaN,$140k + Car Park - Call James Calleja 0430 058...,Real Estate & Property,"Commercial Sales, Leasing & Property Mgmt",Melbourne,Full Time,CBD & Inner Suburbs,Melbourne
4,38856271,Technology Manager | Travel Industry,Rare opportunity for an experienced Technology...,<HTML>This is a key role within a market leadi...,Lead overarching strategy around Technology wi...,You will be responsible for all Technology and...,Competitive Salary package of $110K - $120K + ...,$110k - $120k p.a. + Numerous Perks!,Information & Communication Technology,Management,Auckland,Full Time,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,38836927,Data Analyst,"Data Analyst with Adobe Suite, Analytics and T...",<HTML><strong>The Company</strong><br> I am wo...,NaN,NaN,NaN,NaN,Information & Communication Technology,Business/Systems Analysts,Sydney,Full Time,"CBD, Inner West & Eastern Suburbs",Sydney
49996,38894517,Regional Coordinator - Goldfields,WAPHA > Health Industry - Planning and commiss...,<HTML><p><strong>About Us</strong></p> <p>WA P...,NaN,NaN,NaN,NaN,Healthcare & Medical,Other,Perth,Full Time,NaN,North Perth
49997,38854483,Warehouse Coordinator,Temporary to permanent opportunity with a fant...,"<HTML><div><em>Our client, a Global leader in ...",NaN,NaN,NaN,NaN,"Manufacturing, Transport & Logistics","Warehousing, Storage & Distribution",Melbourne,Full Time,Bayside & South Eastern Suburbs,NaN
49998,38905179,Expression of Interest - Civil Labourer,Our clients are looking for an experienced Civ...,<HTML><p><strong>Expression of Interest - Civi...,NaN,NaN,NaN,NaN,Trades & Services,Labourers,Coffs Harbour & North Coast,Casual/Vacation,NaN,Coffs Harbour


In [32]:
# Because the json_normalise() function retained the JS Object pathnames when it flattened the data frame, 
# we will need to clean up the column names from those nested JS Objects ease of reference.
df = df.rename(columns={'metadata.standout.bullet1': 'bullet1', 
                    'metadata.standout.bullet2': 'bullet2',
                    'metadata.standout.bullet3': 'bullet3',
                    'metadata.additionalSalaryText': 'additionalSalaryText',
                    'metadata.classification.name': 'classification_name',
                    'metadata.subClassification.name': 'subclassification_name',
                    'metadata.location.name': 'location_name',
                    'metadata.workType.name': 'workType',
                    'metadata.area.name': 'areaName',
                    'metadata.suburb.name': 'suburbName'})
df

,id,title,abstract,content,bullet1,bullet2,bullet3,additionalSalaryText,classification_name,subclassification_name,location_name,workType,areaName,suburbName
0,38915469,Recruitment Consultant,We are looking for someone to focus purely on ...,<HTML><p>Are you looking to join a thriving bu...,Join a Sector that is considered Recession Pro...,Excellent opportunity for Career Progression ...,Make a Diference whilst earning Money and havi...,commission,Education & Training,Other,Sydney,Full Time,NaN,NaN
1,38934839,Computers Salesperson - Coburg,Passionate about exceptional customer service?...,<HTML><p>&middot;&nbsp;&nbsp;Casual hours as r...,NaN,NaN,NaN,Attractive Commission - Uncapped Earning Poten...,Retail & Consumer Products,Retail Assistants,Melbourne,Casual/Vacation,Northern Suburbs,Coburg
2,38946054,Senior Developer | SA,Readifarians are known for discovering the lat...,<HTML><p>Readify helps organizations innovate ...,"Design, develop, test and deliver custom softw...",Keep your skills current with 20 x paid profes...,Flexible & inclusive work environment,NaN,Information & Communication Technology,Consultants,Adelaide,Full Time,NaN,NaN
3,38833950,Senior Commercial Property Manager | Leading T...,~ Rare opportunity for a Senior PM to step int...,<HTML><p><strong>WayPoint Recruitment&nbsp;</s...,NaN,NaN,NaN,$140k + Car Park - Call James Calleja 0430 058...,Real Estate & Property,"Commercial Sales, Leasing & Property Mgmt",Melbourne,Full Time,CBD & Inner Suburbs,Melbourne
4,38856271,Technology Manager | Travel Industry,Rare opportunity for an experienced Technology...,<HTML>This is a key role within a market leadi...,Lead overarching strategy around Technology wi...,You will be responsible for all Technology and...,Competitive Salary package of $110K - $120K + ...,$110k - $120k p.a. + Numerous Perks!,Information & Communication Technology,Management,Auckland,Full Time,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,38836927,Data Analyst,"Data Analyst with Adobe Suite, Analytics and T...",<HTML><strong>The Company</strong><br> I am wo...,NaN,NaN,NaN,NaN,Information & Communication Technology,Business/Systems Analysts,Sydney,Full Time,"CBD, Inner West & Eastern Suburbs",Sydney
49996,38894517,Regional Coordinator - Goldfields,WAPHA > Health Industry - Planning and commiss...,<HTML><p><strong>About Us</strong></p> <p>WA P...,NaN,NaN,NaN,NaN,Healthcare & Medical,Other,Perth,Full Time,NaN,North Perth
49997,38854483,Warehouse Coordinator,Temporary to permanent opportunity with a fant...,"<HTML><div><em>Our client, a Global leader in ...",NaN,NaN,NaN,NaN,"Manufacturing, Transport & Logistics","Warehousing, Storage & Distribution",Melbourne,Full Time,Bayside & South Eastern Suburbs,NaN
49998,38905179,Expression of Interest - Civil Labourer,Our clients are looking for an experienced Civ...,<HTML><p><strong>Expression of Interest - Civi...,NaN,NaN,NaN,NaN,Trades & Services,Labourers,Coffs Harbour & North Coast,Casual/Vacation,NaN,Coffs Harbour


## Data Preprocessing

### Perform HTML clean up
<b>Perform a clean up to remove HTML tags from the text using some RegEx and BeautifulSoup. Ordinarily, I'm not too keen on RegEx (especially when taking into account data streaming in a production environment), due to its rigidity and propensity to break when unexpected data is streamed into the model. But, given that we're working with HTML tags which are typical and predictable, I felt it okay to use a large and encompassing expression to remove them from our data.</b>

<b>N.B. I'm making an assumption that the 'abstract' column is what displays on the search results page, and the 'content' is the detailed description of the job requirements after the user clicks into the job ad.</b>
<br><b>&nbspI've also elected to keep the 'id' column in the data frame, as I'll be using this as an index for the recommender once the data preprocessing is complete.</b>

In [33]:
# The following RegEx is fairly encompassing with regards to successfully identifying HTML tags contained in text. 
# Because data streaming is a consideration, this RegEx has been assembled take into account URLs/URIs, 
# even though we don't yet have them in our text. If those elements become present in the future, due to some change in the 
# JSON extract in the future, this RegEx will strip those if they ever become present.

regex = r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))'

def stripTags(x):
    # BeautifulSoup on content
    soup = BeautifulSoup(x, "html.parser")
    # Get all the text out of the html
    text =  soup.get_text()
    # Returning text stripping out all uri's (as our data streams evolve, these JSON exports could contain URIs in the future)
    return re.sub(regex, "", text)

In [34]:
# Run the stripTags() function, parsing the 'content' column as a positional argument, 
# and build a new column with the cleaned text called 'content_cleaned'
df['content_cleaned'] = df['content'].apply(lambda x: stripTags(x))
df

,id,title,abstract,content,bullet1,bullet2,bullet3,additionalSalaryText,classification_name,subclassification_name,location_name,workType,areaName,suburbName,content_cleaned
0,38915469,Recruitment Consultant,We are looking for someone to focus purely on ...,<HTML><p>Are you looking to join a thriving bu...,Join a Sector that is considered Recession Pro...,Excellent opportunity for Career Progression ...,Make a Diference whilst earning Money and havi...,commission,Education & Training,Other,Sydney,Full Time,NaN,NaN,Are you looking to join a thriving business th...
1,38934839,Computers Salesperson - Coburg,Passionate about exceptional customer service?...,<HTML><p>&middot;&nbsp;&nbsp;Casual hours as r...,NaN,NaN,NaN,Attractive Commission - Uncapped Earning Poten...,Retail & Consumer Products,Retail Assistants,Melbourne,Casual/Vacation,Northern Suburbs,Coburg,· Casual hours as required (transition to Par...
2,38946054,Senior Developer | SA,Readifarians are known for discovering the lat...,<HTML><p>Readify helps organizations innovate ...,"Design, develop, test and deliver custom softw...",Keep your skills current with 20 x paid profes...,Flexible & inclusive work environment,NaN,Information & Communication Technology,Consultants,Adelaide,Full Time,NaN,NaN,Readify helps organizations innovate with tech...
3,38833950,Senior Commercial Property Manager | Leading T...,~ Rare opportunity for a Senior PM to step int...,<HTML><p><strong>WayPoint Recruitment&nbsp;</s...,NaN,NaN,NaN,$140k + Car Park - Call James Calleja 0430 058...,Real Estate & Property,"Commercial Sales, Leasing & Property Mgmt",Melbourne,Full Time,CBD & Inner Suburbs,Melbourne,WayPoint Recruitment have partnered up with a ...
4,38856271,Technology Manager | Travel Industry,Rare opportunity for an experienced Technology...,<HTML>This is a key role within a market leadi...,Lead overarching strategy around Technology wi...,You will be responsible for all Technology and...,Competitive Salary package of $110K - $120K + ...,$110k - $120k p.a. + Numerous Perks!,Information & Communication Technology,Management,Auckland,Full Time,NaN,NaN,This is a key role within a market leading Tra...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,38836927,Data Analyst,"Data Analyst with Adobe Suite, Analytics and T...",<HTML><strong>The Company</strong><br> I am wo...,NaN,NaN,NaN,NaN,Information & Communication Technology,Business/Systems Analysts,Sydney,Full Time,"CBD, Inner West & Eastern Suburbs",Sydney,The Company I am working with a highly reputab...
49996,38894517,Regional Coordinator - Goldfields,WAPHA > Health Industry - Planning and commiss...,<HTML><p><strong>About Us</strong></p> <p>WA P...,NaN,NaN,NaN,NaN,Healthcare & Medical,Other,Perth,Full Time,NaN,North Perth,About Us WA Primary Health Alliance (WAPHA) is...
49997,38854483,Warehouse Coordinator,Temporary to permanent opportunity with a fant...,"<HTML><div><em>Our client, a Global leader in ...",NaN,NaN,NaN,NaN,"Manufacturing, Transport & Logistics","Warehousing, Storage & Distribution",Melbourne,Full Time,Bayside & South Eastern Suburbs,NaN,"Our client, a Global leader in the Healthcare ..."
49998,38905179,Expression of Interest - Civil Labourer,Our clients are looking for an experienced Civ...,<HTML><p><strong>Expression of Interest - Civi...,NaN,NaN,NaN,NaN,Trades & Services,Labourers,Coffs Harbour & North Coast,Casual/Vacation,NaN,Coffs Harbour,Expression of Interest - Civil Labourers Lab...


In [35]:
# There are a number of NaNs across the 'bullet1', 'bullet2', 'bullet3', and 'suburbName' columns. 
# Let's remove the NaNs and insert a blank space, as a blank space will not bias our eventual training set.
df = df.fillna(' ')
df

,id,title,abstract,content,bullet1,bullet2,bullet3,additionalSalaryText,classification_name,subclassification_name,location_name,workType,areaName,suburbName,content_cleaned
0,38915469,Recruitment Consultant,We are looking for someone to focus purely on ...,<HTML><p>Are you looking to join a thriving bu...,Join a Sector that is considered Recession Pro...,Excellent opportunity for Career Progression ...,Make a Diference whilst earning Money and havi...,commission,Education & Training,Other,Sydney,Full Time,,,Are you looking to join a thriving business th...
1,38934839,Computers Salesperson - Coburg,Passionate about exceptional customer service?...,<HTML><p>&middot;&nbsp;&nbsp;Casual hours as r...,,,,Attractive Commission - Uncapped Earning Poten...,Retail & Consumer Products,Retail Assistants,Melbourne,Casual/Vacation,Northern Suburbs,Coburg,· Casual hours as required (transition to Par...
2,38946054,Senior Developer | SA,Readifarians are known for discovering the lat...,<HTML><p>Readify helps organizations innovate ...,"Design, develop, test and deliver custom softw...",Keep your skills current with 20 x paid profes...,Flexible & inclusive work environment,,Information & Communication Technology,Consultants,Adelaide,Full Time,,,Readify helps organizations innovate with tech...
3,38833950,Senior Commercial Property Manager | Leading T...,~ Rare opportunity for a Senior PM to step int...,<HTML><p><strong>WayPoint Recruitment&nbsp;</s...,,,,$140k + Car Park - Call James Calleja 0430 058...,Real Estate & Property,"Commercial Sales, Leasing & Property Mgmt",Melbourne,Full Time,CBD & Inner Suburbs,Melbourne,WayPoint Recruitment have partnered up with a ...
4,38856271,Technology Manager | Travel Industry,Rare opportunity for an experienced Technology...,<HTML>This is a key role within a market leadi...,Lead overarching strategy around Technology wi...,You will be responsible for all Technology and...,Competitive Salary package of $110K - $120K + ...,$110k - $120k p.a. + Numerous Perks!,Information & Communication Technology,Management,Auckland,Full Time,,,This is a key role within a market leading Tra...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,38836927,Data Analyst,"Data Analyst with Adobe Suite, Analytics and T...",<HTML><strong>The Company</strong><br> I am wo...,,,,,Information & Communication Technology,Business/Systems Analysts,Sydney,Full Time,"CBD, Inner West & Eastern Suburbs",Sydney,The Company I am working with a highly reputab...
49996,38894517,Regional Coordinator - Goldfields,WAPHA > Health Industry - Planning and commiss...,<HTML><p><strong>About Us</strong></p> <p>WA P...,,,,,Healthcare & Medical,Other,Perth,Full Time,,North Perth,About Us WA Primary Health Alliance (WAPHA) is...
49997,38854483,Warehouse Coordinator,Temporary to permanent opportunity with a fant...,"<HTML><div><em>Our client, a Global leader in ...",,,,,"Manufacturing, Transport & Logistics","Warehousing, Storage & Distribution",Melbourne,Full Time,Bayside & South Eastern Suburbs,,"Our client, a Global leader in the Healthcare ..."
49998,38905179,Expression of Interest - Civil Labourer,Our clients are looking for an experienced Civ...,<HTML><p><strong>Expression of Interest - Civi...,,,,,Trades & Services,Labourers,Coffs Harbour & North Coast,Casual/Vacation,,Coffs Harbour,Expression of Interest - Civil Labourers Lab...


### Remove punctuation

In [36]:
# Remove punctuation using the 'string.punctuation' library. By using a standard Python corpus of English punctuation, 
# we can run matches that iterate through a loop for two columns that contain punctuation - 'abstract' and 'content'.
def remove_punct(text):
    text_nopunct = "".join([char for char in text if char not in string.punctuation])
    return text_nopunct

df['title_nopunct'] = df['title'].apply(lambda x: remove_punct(x))
df['content_nopunct'] = df['content_cleaned'].apply(lambda x: remove_punct(x))
df['abstract_nopunct'] = df['abstract'].apply(lambda x:remove_punct(x))
df['bullet1_nopunct'] = df['bullet1'].apply(lambda x: remove_punct(x))
df['bullet2_nopunct'] = df['bullet2'].apply(lambda x: remove_punct(x))
df['bullet3_nopunct'] = df['bullet3'].apply(lambda x: remove_punct(x))
df['suburbName_nopunct'] = df['suburbName'].apply(lambda x: remove_punct(x))
df

,id,title,abstract,content,bullet1,bullet2,bullet3,additionalSalaryText,classification_name,subclassification_name,...,areaName,suburbName,content_cleaned,title_nopunct,content_nopunct,abstract_nopunct,bullet1_nopunct,bullet2_nopunct,bullet3_nopunct,suburbName_nopunct
0,38915469,Recruitment Consultant,We are looking for someone to focus purely on ...,<HTML><p>Are you looking to join a thriving bu...,Join a Sector that is considered Recession Pro...,Excellent opportunity for Career Progression ...,Make a Diference whilst earning Money and havi...,commission,Education & Training,Other,...,,,Are you looking to join a thriving business th...,Recruitment Consultant,Are you looking to join a thriving business th...,We are looking for someone to focus purely on ...,Join a Sector that is considered Recession Proof,Excellent opportunity for Career Progression ...,Make a Diference whilst earning Money and havi...,
1,38934839,Computers Salesperson - Coburg,Passionate about exceptional customer service?...,<HTML><p>&middot;&nbsp;&nbsp;Casual hours as r...,,,,Attractive Commission - Uncapped Earning Poten...,Retail & Consumer Products,Retail Assistants,...,Northern Suburbs,Coburg,· Casual hours as required (transition to Par...,Computers Salesperson Coburg,· Casual hours as required transition to Part...,Passionate about exceptional customer service ...,,,,Coburg
2,38946054,Senior Developer | SA,Readifarians are known for discovering the lat...,<HTML><p>Readify helps organizations innovate ...,"Design, develop, test and deliver custom softw...",Keep your skills current with 20 x paid profes...,Flexible & inclusive work environment,,Information & Communication Technology,Consultants,...,,,Readify helps organizations innovate with tech...,Senior Developer SA,Readify helps organizations innovate with tech...,Readifarians are known for discovering the lat...,Design develop test and deliver custom softwar...,Keep your skills current with 20 x paid profes...,Flexible inclusive work environment,
3,38833950,Senior Commercial Property Manager | Leading T...,~ Rare opportunity for a Senior PM to step int...,<HTML><p><strong>WayPoint Recruitment&nbsp;</s...,,,,$140k + Car Park - Call James Calleja 0430 058...,Real Estate & Property,"Commercial Sales, Leasing & Property Mgmt",...,CBD & Inner Suburbs,Melbourne,WayPoint Recruitment have partnered up with a ...,Senior Commercial Property Manager Leading Ti...,WayPoint Recruitment have partnered up with a ...,Rare opportunity for a Senior PM to step into...,,,,Melbourne
4,38856271,Technology Manager | Travel Industry,Rare opportunity for an experienced Technology...,<HTML>This is a key role within a market leadi...,Lead overarching strategy around Technology wi...,You will be responsible for all Technology and...,Competitive Salary package of $110K - $120K + ...,$110k - $120k p.a. + Numerous Perks!,Information & Communication Technology,Management,...,,,This is a key role within a market leading Tra...,Technology Manager Travel Industry,This is a key role within a market leading Tra...,Rare opportunity for an experienced Technology...,Lead overarching strategy around Technology wi...,You will be responsible for all Technology and...,Competitive Salary package of 110K 120K Perks,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,38836927,Data Analyst,"Data Analyst with Adobe Suite, Analytics and T...",<HTML><strong>The Company</strong><br> I am wo...,,,,,Information & Communication Technology,Business/Systems Analysts,...,"CBD, Inner West & Eastern Suburbs",Sydney,The Company I am working with a highly reputab...,Data Analyst,The Company I am working with a highly reputab...,Data Analyst with Adobe Suite Analytics and Ta...,,,,Sydney
49996,38894517,Regional Coordinator - Goldfields,WAPHA > Health Industry - Planning and commiss...,<HTML><p><strong>About Us</strong></p> <p>WA P...,,,,,Healthcare & Medical,Other,...,,North Perth,About Us WA Primary Health Alliance 

### Tokenization

In [37]:
# For tokenization, we'll need to use a little RegEx so as to split words into individual tokens our six columns of interest. 
# We'll also be reducing all our tokens into lowercase characters.
def tokenize(text):
    tokens = re.split('\W+', text)
    return tokens

df['title_tokenized'] = df['title_nopunct'].apply(lambda x: tokenize(x.lower()))
df['content_tokenized'] = df['content_nopunct'].apply(lambda x: tokenize(x.lower()))
df['abstract_tokenized'] = df['abstract_nopunct'].apply(lambda x: tokenize(x.lower()))
df['bullet1_tokenized'] = df['bullet1_nopunct'].apply(lambda x: tokenize(x.lower()))
df['bullet2_tokenized'] = df['bullet2_nopunct'].apply(lambda x: tokenize(x.lower()))
df['bullet3_tokenized'] = df['bullet3_nopunct'].apply(lambda x: tokenize(x.lower()))
df['suburbName_tokenized'] = df['suburbName_nopunct'].apply(lambda x: tokenize(x.lower()))
df

,id,title,abstract,content,bullet1,bullet2,bullet3,additionalSalaryText,classification_name,subclassification_name,...,bullet2_nopunct,bullet3_nopunct,suburbName_nopunct,title_tokenized,content_tokenized,abstract_tokenized,bullet1_tokenized,bullet2_tokenized,bullet3_tokenized,suburbName_tokenized
0,38915469,Recruitment Consultant,We are looking for someone to focus purely on ...,<HTML><p>Are you looking to join a thriving bu...,Join a Sector that is considered Recession Pro...,Excellent opportunity for Career Progression ...,Make a Diference whilst earning Money and havi...,commission,Education & Training,Other,...,Excellent opportunity for Career Progression ...,Make a Diference whilst earning Money and havi...,,"[recruitment, consultant]","[are, you, looking, to, join, a, thriving, bus...","[we, are, looking, for, someone, to, focus, pu...","[join, a, sector, that, is, considered, recess...","[excellent, opportunity, for, career, progress...","[make, a, diference, whilst, earning, money, a...","[, ]"
1,38934839,Computers Salesperson - Coburg,Passionate about exceptional customer service?...,<HTML><p>&middot;&nbsp;&nbsp;Casual hours as r...,,,,Attractive Commission - Uncapped Earning Poten...,Retail & Consumer Products,Retail Assistants,...,,,Coburg,"[computers, salesperson, coburg]","[, casual, hours, as, required, transition, to...","[passionate, about, exceptional, customer, ser...","[, ]","[, ]","[, ]",[coburg]
2,38946054,Senior Developer | SA,Readifarians are known for discovering the lat...,<HTML><p>Readify helps organizations innovate ...,"Design, develop, test and deliver custom softw...",Keep your skills current with 20 x paid profes...,Flexible & inclusive work environment,,Information & Communication Technology,Consultants,...,Keep your skills current with 20 x paid profes...,Flexible inclusive work environment,,"[senior, developer, sa]","[readify, helps, organizations, innovate, with...","[readifarians, are, known, for, discovering, t...","[design, develop, test, and, deliver, custom, ...","[keep, your, skills, current, with, 20, x, pai...","[flexible, inclusive, work, environment]","[, ]"
3,38833950,Senior Commercial Property Manager | Leading T...,~ Rare opportunity for a Senior PM to step int...,<HTML><p><strong>WayPoint Recruitment&nbsp;</s...,,,,$140k + Car Park - Call James Calleja 0430 058...,Real Estate & Property,"Commercial Sales, Leasing & Property Mgmt",...,,,Melbourne,"[senior, commercial, property, manager, leadin...","[waypoint, recruitment, have, partnered, up, w...","[, rare, opportunity, for, a, senior, pm, to, ...","[, ]","[, ]","[, ]",[melbourne]
4,38856271,Technology Manager | Travel Industry,Rare opportunity for an experienced Technology...,<HTML>This is a key role within a market leadi...,Lead overarching strategy around Technology wi...,You will be responsible for all Technology and...,Competitive Salary package of $110K - $120K + ...,$110k - $120k p.a. + Numerous Perks!,Information & Communication Technology,Management,...,You will be responsible for all Technology and...,Competitive Salary package of 110K 120K Perks,,"[technology, manager, travel, industry]","[this, is, a, key, role, within, a, market, le...","[rare, opportunity, for, an, experienced, tech...","[lead, overarching, strategy, around, technolo...","[you, will, be, responsible, for, all, technol...","[competitive, salary, package, of, 110k, 120k,...","[, ]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,38836927,Data Analyst,"Data Analyst with Adobe Suite, Analytics and T...",<HTML><strong>The Company</strong><br> I am wo...,,,,,Information & Communication Technology,Business/Systems Analysts,...,,,Sydney,"[data, analyst]","[the, company, i, am, working, with, a, highly...","[data, analyst, with, adobe, suite, analytics,...","[, ]","[, ]","[, ]",[sydney]
49996,38894517,Regional Coordinator - Goldfields,WAPHA > Health Industry - Planning and commiss...,<HTML><p><strong>About Us</s

### Remove Stop words

In [38]:
# For our ML development, we'll also need to remove all the stopwords from our corpus
stopword = corpus.stopwords.words('English')

def remove_stopwords(tokenized_list):
    text = [word for word in tokenized_list if word not in stopword]
    return text

df['title_nostop'] = df['title_tokenized'].apply(lambda x: remove_stopwords(x))
df['content_nostop'] = df['content_tokenized'].apply(lambda x: remove_stopwords(x))
df['abstract_nostop'] = df['abstract_tokenized'].apply(lambda x: remove_stopwords(x))
df['bullet1_nostop'] = df['bullet1_tokenized'].apply(lambda x: remove_stopwords(x))
df['bullet2_nostop'] = df['bullet2_tokenized'].apply(lambda x: remove_stopwords(x))
df['bullet3_nostop'] = df['bullet3_tokenized'].apply(lambda x: remove_stopwords(x))
df['suburbName_nostop'] = df['suburbName_tokenized'].apply(lambda x: remove_stopwords(x))
df

,id,title,abstract,content,bullet1,bullet2,bullet3,additionalSalaryText,classification_name,subclassification_name,...,bullet2_tokenized,bullet3_tokenized,suburbName_tokenized,title_nostop,content_nostop,abstract_nostop,bullet1_nostop,bullet2_nostop,bullet3_nostop,suburbName_nostop
0,38915469,Recruitment Consultant,We are looking for someone to focus purely on ...,<HTML><p>Are you looking to join a thriving bu...,Join a Sector that is considered Recession Pro...,Excellent opportunity for Career Progression ...,Make a Diference whilst earning Money and havi...,commission,Education & Training,Other,...,"[excellent, opportunity, for, career, progress...","[make, a, diference, whilst, earning, money, a...","[, ]","[recruitment, consultant]","[looking, join, thriving, business, reward, ha...","[looking, someone, focus, purely, recruiting, ...","[join, sector, considered, recession, proof, ]","[excellent, opportunity, career, progression, ...","[make, diference, whilst, earning, money, fun]","[, ]"
1,38934839,Computers Salesperson - Coburg,Passionate about exceptional customer service?...,<HTML><p>&middot;&nbsp;&nbsp;Casual hours as r...,,,,Attractive Commission - Uncapped Earning Poten...,Retail & Consumer Products,Retail Assistants,...,"[, ]","[, ]",[coburg],"[computers, salesperson, coburg]","[, casual, hours, required, transition, partti...","[passionate, exceptional, customer, service, e...","[, ]","[, ]","[, ]",[coburg]
2,38946054,Senior Developer | SA,Readifarians are known for discovering the lat...,<HTML><p>Readify helps organizations innovate ...,"Design, develop, test and deliver custom softw...",Keep your skills current with 20 x paid profes...,Flexible & inclusive work environment,,Information & Communication Technology,Consultants,...,"[keep, your, skills, current, with, 20, x, pai...","[flexible, inclusive, work, environment]","[, ]","[senior, developer, sa]","[readify, helps, organizations, innovate, tech...","[readifarians, known, discovering, latest, gre...","[design, develop, test, deliver, custom, softw...","[keep, skills, current, 20, x, paid, professio...","[flexible, inclusive, work, environment]","[, ]"
3,38833950,Senior Commercial Property Manager | Leading T...,~ Rare opportunity for a Senior PM to step int...,<HTML><p><strong>WayPoint Recruitment&nbsp;</s...,,,,$140k + Car Park - Call James Calleja 0430 058...,Real Estate & Property,"Commercial Sales, Leasing & Property Mgmt",...,"[, ]","[, ]",[melbourne],"[senior, commercial, property, manager, leadin...","[waypoint, recruitment, partnered, respected, ...","[, rare, opportunity, senior, pm, step, respec...","[, ]","[, ]","[, ]",[melbourne]
4,38856271,Technology Manager | Travel Industry,Rare opportunity for an experienced Technology...,<HTML>This is a key role within a market leadi...,Lead overarching strategy around Technology wi...,You will be responsible for all Technology and...,Competitive Salary package of $110K - $120K + ...,$110k - $120k p.a. + Numerous Perks!,Information & Communication Technology,Management,...,"[you, will, be, responsible, for, all, technol...","[competitive, salary, package, of, 110k, 120k,...","[, ]","[technology, manager, travel, industry]","[key, role, within, market, leading, travel, c...","[rare, opportunity, experienced, technology, m...","[lead, overarching, strategy, around, technolo...","[responsible, technology, systems, development...","[competitive, salary, package, 110k, 120k, perks]","[, ]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,38836927,Data Analyst,"Data Analyst with Adobe Suite, Analytics and T...",<HTML><strong>The Company</strong><br> I am wo...,,,,,Information & Communication Technology,Business/Systems Analysts,...,"[, ]","[, ]",[sydney],"[data, analyst]","[company, working, highly, reputable, insuranc...","[data, analyst, adobe, suite, analytics, targe...","[, ]","[, ]","[, ]",[sydney]
49996,38894517,Regional Coordinator - Goldfields,WAPHA > Health In

### Lemmatization
<b>To assist our NLP algorithm with infering the meanings contained in words, I've chosen to use the WordNet lemmatizer in the NLTK library. Since, the 'content' and 'abstract' columns are the two columns with the largest corpus of full text sentences and paragraphs, it makes sense to take the time to extract as much text <i>information</i> (over just a bag of words approach) as possible. I far prefer lemmatization to stemming, as stemming employs a heuristic technique of chopping off words at their stems, whereas lemmatization is capable of inferring the actual meaning behind the words themselves - in our case, we can use the WordNet library to help us with this process. The only drawback to lemmatization is that its computationally slower, but much more accurate. In my mind, the tradeoff for higher accuracy to slightly slower computation is worthwhile.</b>

In [39]:
# Use the Word Net Lemmatizer in the nltk library
wn = WordNetLemmatizer()

def lemmatizing(tokenized_text):
    text = [wn.lemmatize(word) for word in tokenized_text]
    return text

df['title_lemmatized'] = df['title_nostop'].apply(lambda x: lemmatizing(x))
df['content_lemmatized'] = df['content_nostop'].apply(lambda x: lemmatizing(x))
df['abstract_lemmatized'] = df['abstract_nostop'].apply(lambda x: lemmatizing(x))
df['bullet1_lemmatized'] = df['bullet1_nostop'].apply(lambda x: lemmatizing(x))
df['bullet2_lemmatized'] = df['bullet2_nostop'].apply(lambda x: lemmatizing(x))
df['bullet3_lemmatized'] = df['bullet3_nostop'].apply(lambda x: lemmatizing(x))
df['suburbName_lemmatized'] = df['suburbName_nostop'].apply(lambda x: lemmatizing(x))
df

,id,title,abstract,content,bullet1,bullet2,bullet3,additionalSalaryText,classification_name,subclassification_name,...,bullet2_nostop,bullet3_nostop,suburbName_nostop,title_lemmatized,content_lemmatized,abstract_lemmatized,bullet1_lemmatized,bullet2_lemmatized,bullet3_lemmatized,suburbName_lemmatized
0,38915469,Recruitment Consultant,We are looking for someone to focus purely on ...,<HTML><p>Are you looking to join a thriving bu...,Join a Sector that is considered Recession Pro...,Excellent opportunity for Career Progression ...,Make a Diference whilst earning Money and havi...,commission,Education & Training,Other,...,"[excellent, opportunity, career, progression, ...","[make, diference, whilst, earning, money, fun]","[, ]","[recruitment, consultant]","[looking, join, thriving, business, reward, ha...","[looking, someone, focus, purely, recruiting, ...","[join, sector, considered, recession, proof, ]","[excellent, opportunity, career, progression, ...","[make, diference, whilst, earning, money, fun]","[, ]"
1,38934839,Computers Salesperson - Coburg,Passionate about exceptional customer service?...,<HTML><p>&middot;&nbsp;&nbsp;Casual hours as r...,,,,Attractive Commission - Uncapped Earning Poten...,Retail & Consumer Products,Retail Assistants,...,"[, ]","[, ]",[coburg],"[computer, salesperson, coburg]","[, casual, hour, required, transition, parttim...","[passionate, exceptional, customer, service, e...","[, ]","[, ]","[, ]",[coburg]
2,38946054,Senior Developer | SA,Readifarians are known for discovering the lat...,<HTML><p>Readify helps organizations innovate ...,"Design, develop, test and deliver custom softw...",Keep your skills current with 20 x paid profes...,Flexible & inclusive work environment,,Information & Communication Technology,Consultants,...,"[keep, skills, current, 20, x, paid, professio...","[flexible, inclusive, work, environment]","[, ]","[senior, developer, sa]","[readify, help, organization, innovate, techno...","[readifarians, known, discovering, latest, gre...","[design, develop, test, deliver, custom, softw...","[keep, skill, current, 20, x, paid, profession...","[flexible, inclusive, work, environment]","[, ]"
3,38833950,Senior Commercial Property Manager | Leading T...,~ Rare opportunity for a Senior PM to step int...,<HTML><p><strong>WayPoint Recruitment&nbsp;</s...,,,,$140k + Car Park - Call James Calleja 0430 058...,Real Estate & Property,"Commercial Sales, Leasing & Property Mgmt",...,"[, ]","[, ]",[melbourne],"[senior, commercial, property, manager, leadin...","[waypoint, recruitment, partnered, respected, ...","[, rare, opportunity, senior, pm, step, respec...","[, ]","[, ]","[, ]",[melbourne]
4,38856271,Technology Manager | Travel Industry,Rare opportunity for an experienced Technology...,<HTML>This is a key role within a market leadi...,Lead overarching strategy around Technology wi...,You will be responsible for all Technology and...,Competitive Salary package of $110K - $120K + ...,$110k - $120k p.a. + Numerous Perks!,Information & Communication Technology,Management,...,"[responsible, technology, systems, development...","[competitive, salary, package, 110k, 120k, perks]","[, ]","[technology, manager, travel, industry]","[key, role, within, market, leading, travel, c...","[rare, opportunity, experienced, technology, m...","[lead, overarching, strategy, around, technolo...","[responsible, technology, system, development,...","[competitive, salary, package, 110k, 120k, perk]","[, ]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,38836927,Data Analyst,"Data Analyst with Adobe Suite, Analytics and T...",<HTML><strong>The Company</strong><br> I am wo...,,,,,Information & Communication Technology,Business/Systems Analysts,...,"[, ]","[, ]",[sydney],"[data, analyst]","[company, working, highly, reputable, insuranc...","[data, analyst, adobe, suite, analytics, targe...","[, ]","[, ]","[, ]",[sydney]
49996,38894517,Regional Coordinator - Goldfields,WAP

In [56]:
final_df = df[['id','content_cleaned', 'bullet1', 'bullet2', 'bullet3', 'title_lemmatized', 'content_lemmatized', 
               'abstract_lemmatized', 'bullet1_lemmatized', 'bullet2_lemmatized', 'bullet3_lemmatized', 'suburbName_lemmatized']]

In [57]:
# Combine the final preprocessed data into 'keywords' column. The concatenation requires the dtypes to be cast into strings, 
# otherwise the interpreter will throw errors.
final_df['keywords'] = final_df['title_lemmatized'].map(str) + ' ' + final_df['content_lemmatized'].map(str) + ' ' 
+ final_df['abstract_lemmatized'].map(str) + ' ' + final_df['bullet1_lemmatized'].map(str) + ' ' 
+ final_df['bullet2_lemmatized'].map(str) + ' ' + final_df['bullet3_lemmatized'].map(str) + ' ' 
+ final_df['suburbName_lemmatized'].map(str)

final_df

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id,content_cleaned,bullet1,bullet2,bullet3,title_lemmatized,content_lemmatized,abstract_lemmatized,bullet1_lemmatized,bullet2_lemmatized,bullet3_lemmatized,suburbName_lemmatized,keywords
0,38915469,Are you looking to join a thriving business th...,Join a Sector that is considered Recession Pro...,Excellent opportunity for Career Progression ...,Make a Diference whilst earning Money and havi...,"[recruitment, consultant]","[looking, join, thriving, business, reward, ha...","[looking, someone, focus, purely, recruiting, ...","[join, sector, considered, recession, proof, ]","[excellent, opportunity, career, progression, ...","[make, diference, whilst, earning, money, fun]","[, ]","['recruitment', 'consultant'] ['looking', 'joi..."
1,38934839,· Casual hours as required (transition to Par...,,,,"[computer, salesperson, coburg]","[, casual, hour, required, transition, parttim...","[passionate, exceptional, customer, service, e...","[, ]","[, ]","[, ]",[coburg],"['computer', 'salesperson', 'coburg'] ['', 'ca..."
2,38946054,Readify helps organizations innovate with tech...,"Design, develop, test and deliver custom softw...",Keep your skills current with 20 x paid profes...,Flexible & inclusive work environment,"[senior, developer, sa]","[readify, help, organization, innovate, techno...","[readifarians, known, discovering, latest, gre...","[design, develop, test, deliver, custom, softw...","[keep, skill, current, 20, x, paid, profession...","[flexible, inclusive, work, environment]","[, ]","['senior', 'developer', 'sa'] ['readify', 'hel..."
3,38833950,WayPoint Recruitment have partnered up with a ...,,,,"[senior, commercial, property, manager, leadin...","[waypoint, recruitment, partnered, respected, ...","[, rare, opportunity, senior, pm, step, respec...","[, ]","[, ]","[, ]",[melbourne],"['senior', 'commercial', 'property', 'manager'..."
4,38856271,This is a key role within a market leading Tra...,Lead overarching strategy around Technology wi...,You will be responsible for all Technology and...,Competitive Salary package of $110K - $120K + ...,"[technology, manager, travel, industry]","[key, role, within, market, leading, travel, c...","[rare, opportunity, experienced, technology, m...","[lead, overarching, strategy, around, technolo...","[responsible, technology, system, development,...","[competitive, salary, package, 110k, 120k, perk]","[, ]","['technology', 'manager', 'travel', 'industry'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,38836927,The Company I am working with a highly reputab...,,,,"[data, analyst]","[company, working, highly, reputable, insuranc...","[data, analyst, adobe, suite, analytics, targe...","[, ]","[, ]","[, ]",[sydney],"['data', 'analyst'] ['company', 'working', 'hi..."
49996,38894517,About Us WA Primary Health Alliance (WAPHA) is...,,,,"[regional, coordinator, goldfield]","[u, wa, primary, health, alliance, wapha, peak...","[wapha, health, industry, planning, commission...","[, ]","[, ]","[, ]","[north, perth]","['regional', 'coordinator', 'goldfield'] ['u',..."
49997,38854483,"Our client, a Global leader in the Healthcare ...",,,,"[warehouse, coordinator]","[client, global, leader, healthcare, industry,...","[temporary, permanent, opportunity, fantastic,...","[, ]","[, ]","[, ]","[, ]","['warehouse', 'coordinator'] ['client', 'globa..."
49998,38905179,Expression of Interest - Civil Labourers Lab...,,,,"[expression, interest, civil, labourer]","[expression, interest, civil, labourer, labour...","[client, looking, experienced, civil, labourer...","[, ]","[, ]","[, ]","[coffs, harbour]","['expression', 'interest', 'civil', 'labourer'..."


In [58]:
# Select the preprocessed text columns to begin the final phase of preprocessing - the Bag of Words
final_df['bag_of_words'] = ''
columns = ['title_lemmatized', 'content_lemmatized', 'abstract_lemmatized', 'bullet1_lemmatized', 'bullet2_lemmatized', 
           'bullet3_lemmatized', 'suburbName_lemmatized']
for index, row in final_df.iterrows():
    words = ''
    for col in columns:
        words += ' '.join(row[col]) + ' '
    row['bag_of_words'] = words

final_df[['id', 'title_lemmatized', 'content_cleaned', 'abstract_lemmatized', 'bullet1', 'bullet2', 'bullet3', 'bag_of_words']]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id,title_lemmatized,content_cleaned,abstract_lemmatized,bullet1,bullet2,bullet3,bag_of_words
0,38915469,"[recruitment, consultant]",Are you looking to join a thriving business th...,"[looking, someone, focus, purely, recruiting, ...",Join a Sector that is considered Recession Pro...,Excellent opportunity for Career Progression ...,Make a Diference whilst earning Money and havi...,recruitment consultant looking join thriving b...
1,38934839,"[computer, salesperson, coburg]",· Casual hours as required (transition to Par...,"[passionate, exceptional, customer, service, e...",,,,computer salesperson coburg casual hour requi...
2,38946054,"[senior, developer, sa]",Readify helps organizations innovate with tech...,"[readifarians, known, discovering, latest, gre...","Design, develop, test and deliver custom softw...",Keep your skills current with 20 x paid profes...,Flexible & inclusive work environment,senior developer sa readify help organization ...
3,38833950,"[senior, commercial, property, manager, leadin...",WayPoint Recruitment have partnered up with a ...,"[, rare, opportunity, senior, pm, step, respec...",,,,senior commercial property manager leading tie...
4,38856271,"[technology, manager, travel, industry]",This is a key role within a market leading Tra...,"[rare, opportunity, experienced, technology, m...",Lead overarching strategy around Technology wi...,You will be responsible for all Technology and...,Competitive Salary package of $110K - $120K + ...,technology manager travel industry key role wi...
...,...,...,...,...,...,...,...,...
49995,38836927,"[data, analyst]",The Company I am working with a highly reputab...,"[data, analyst, adobe, suite, analytics, targe...",,,,data analyst company working highly reputable ...
49996,38894517,"[regional, coordinator, goldfield]",About Us WA Primary Health Alliance (WAPHA) is...,"[wapha, health, industry, planning, commission...",,,,regional coordinator goldfield u wa primary he...
49997,38854483,"[warehouse, coordinator]","Our client, a Global leader in the Healthcare ...","[temporary, permanent, opportunity, fantastic,...",,,,warehouse coordinator client global leader hea...
49998,38905179,"[expression, interest, civil, labourer]",Expression of Interest - Civil Labourers Lab...,"[client, looking, experienced, civil, labourer...",,,,expression interest civil labourer expression ...


In [92]:
#final_df.reset_index('id')
final_df.set_index('title_lemmatized', inplace=True)

## The Recommender Systems and their evaluations of accuracy
<b>N.B. In my view, this type of business problem requires a recommender system. Whilst, there are a number of approaches to recommender systems, I feel that Cosine Similarity, using TF-IDF and Count Vectoriser as feature extraction methods, are the most popular and useful algorithms.</b>
<br><br></b> I'll also be using K-Nearest Neighbours for the recommender. KNNs are actually remarkably powerful in this context as well (using the same TF-IDF and Count Vectorisers as feature extraction techniques) due to their reliance on the Bayes Error Rate (a powerful form irreducable error in Bayesian Statistics). On this same note, with KNNs, the training data is based around a Probability Distribution over just the Cosine angle divided by the lengths of A & B (as it is with Cosine Similarity). As such, I'll be looking to also use KNNs to compare & evaluate our model performance.

<b>An alternative to using TF-IDF is to use 'Coreference Resolution' (https://neurosys.com/popular-frameworks-coreference-resolution/) on the text. This can reveal some more interesting features that TF-IDF tends to miss.</b>

### Cosine Similarity using TF-IDF

In [94]:
# Perform feature extraction using TF-IDF
user_tfidf = TfidfVectorizer(analyzer='word')
tfidf_matrix = user_tfidf.fit_transform((final_df['bag_of_words']))
tfidf_matrix

<50000x221329 sparse matrix of type '<class 'numpy.float64'>'
	with 7333350 stored elements in Compressed Sparse Row format>

In [95]:
# Converting to a Pandas Series and index so as to match indexes with text
indices = pd.Series(final_df.index)

In [96]:
indices

0                                [recruitment, consultant]
1                          [computer, salesperson, coburg]
2                                  [senior, developer, sa]
3        [senior, commercial, property, manager, leadin...
4                  [technology, manager, travel, industry]
                               ...                        
49995                                      [data, analyst]
49996                   [regional, coordinator, goldfield]
49997                             [warehouse, coordinator]
49998              [expression, interest, civil, labourer]
49999                              [provisioning, support]
Name: title_lemmatized, Length: 50000, dtype: object

In [73]:
# Calculate Cosine Similarity for TF-IDF
cosine_sim_tfidf = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim_tfidf

array([[1.        , 0.05456743, 0.05314665, ..., 0.05962288, 0.02971213,
        0.0438587 ],
       [0.05456743, 1.        , 0.07267657, ..., 0.0492142 , 0.01575236,
        0.06332058],
       [0.05314665, 0.07267657, 1.        , ..., 0.03888358, 0.02704931,
        0.07209452],
       ...,
       [0.05962288, 0.0492142 , 0.03888358, ..., 1.        , 0.0534741 ,
        0.05076195],
       [0.02971213, 0.01575236, 0.02704931, ..., 0.0534741 , 1.        ,
        0.01418496],
       [0.0438587 , 0.06332058, 0.07209452, ..., 0.05076195, 0.01418496,
        1.        ]])

In [76]:
# Sort the cosine similarity array in descending order
cosine_output = list(cosine_sim_tfidf)
cosine_output.sort(key=lambda x:x[1], reverse=True)

In [109]:
# Create function for parsing recommendations from the Cosine Similarity array into readable text
def recommender_tfidf(text, cosine_sim = cosine_sim_tfidf):
    
    recommended_text = []

    # Grab the index of the product that matches the product_name
    idx = indices[indices == text].index[1]

    # Grab the top 5 most similar text strings, but we'll start at the second value so as not to match with itself
    top_5_text = list(cosine_sim[1:6]) 

    # Populate the list with the top 5 recommended text strings
    for i in top_5_text:
        recommended_text.append(list(final_df.index))
    
    return recommended_text

In [99]:
def recommender_tfidf(title, cosine_sim = cosine_sim_tfidf):
    
    index = indices[indices == title].index[0]
    similarity_scores = pd.Series(cosine_sim[index]).sort_values(ascending = False)
    top_10_text = list(similarity_scores.iloc[1:11].index)
    recommended_text = [list(final_df.index)[i] for i in top_10_movies]

    return recommended_movies

In [110]:
recommender_tfidf('accountant')

IndexError: index 1 is out of bounds for axis 0 with size 0

### Cosine Similarity using Count Vectoriser

## K-Nearest Neighbours (using TF-IDF and Count Vectoriser for feature extraction)